Q1. You are provided with data with 10-mins, 60-mins and 1-day resolution (Merge.csv) 

Please merge them into a pandas Dataframe with 2-hours resolution in between 7:00 – 17:00 only as index. 

Please take the average of the 10-mins and 60-mins resolution prices during the 2-hours window and forward fill the 1-day resolution prices in the 2-hours window.

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
import pandas as pd

In [3]:
# Read csv file and set the datetime column as index
df = pd.read_csv('data_files/Merge.csv', parse_dates=True, index_col='Datetime')

In [4]:
# Convert the index column into datetime
df.index = pd.to_datetime(df.index, dayfirst=True)

In [5]:
df.head()

,Resolution,Price
Datetime,,
2021-11-01 07:00:00,10MIN,70.000
2021-11-01 07:10:00,10MIN,NaN
2021-11-01 07:20:00,10MIN,71.500
2021-11-01 07:30:00,10MIN,71.375
2021-11-01 07:40:00,10MIN,NaN


 ### 2-hour windows   

In [6]:
# Groupby the resolution as well as the date to get 2-hour window average between 7am - 5pm.
# reset_index() will keep only Datetime column as index.
# df.index.normalize() creates a new column with the same name as the index, therefore it is renamed to 'Date'.
df_2h_window = df.groupby(['Resolution', df.index.normalize()]).resample('2H', origin='start')\
.mean(numeric_only=True).reset_index(level=[0,1]).rename(columns={'Datetime':'Date'})


In [7]:
df_2h_window

,Resolution,Date,Price
Datetime,,,
2021-11-01 07:00:00,10MIN,2021-11-01,70.857500
2021-11-01 09:00:00,10MIN,2021-11-01,71.566667
2021-11-01 11:00:00,10MIN,2021-11-01,68.812500
2021-11-01 13:00:00,10MIN,2021-11-01,63.250000
2021-11-01 15:00:00,10MIN,2021-11-01,65.589583
...,...,...,...
2021-11-26 00:00:00,D,2021-11-26,88.350000
2021-11-27 00:00:00,D,2021-11-27,NaN
2021-11-28 00:00:00,D,2021-11-28,NaN


In [8]:
# Split the final dataframe with 2-hour averages into 10MIN, 1H, 1D resolution.
# Extra columns are dropped from each dataframe.
df_10min = df_2h_window.loc[df_2h_window['Resolution']=='10MIN'].drop(['Resolution', 'Date'], axis=1)
df_1H = df_2h_window.loc[df_2h_window['Resolution']=='1H'].drop('Resolution', axis=1)
df_1D = df_2h_window.loc[df_2h_window['Resolution']=='D'].drop('Resolution', axis=1)

In [9]:
# Merge the 10min and 1h dataframes dataframes into a wide format using the index of each.
# The price columns are renamed to 10MIN and 1H respectively.
# The 1H rows have time as 7, 9, 11 and for 10 min rows the values of the hourly prices are carried forward.
tensixty_df = df_10min.merge(df_1H, left_index=True, right_index=True, how='left')\
.rename(columns={'Price_x':'10MIN', 'Price_y':'1H'})

In [10]:
# Merge the day resolution into the dataframe with forward filling.
# The 1D indices are only dates so they are merged based on index of df_1D and index of tensixty_df.
# ffill() forward fills the NaN values if any.
final_df = tensixty_df.merge(df_1D, right_index=True, left_on='Date', how='left')\
.drop(['Date_x', 'Date_y'], axis=1).rename(columns={'Price':'1D'}).ffill()

In [11]:
# View final dataframe
final_df

,10MIN,1H,1D
Datetime,,,
2021-11-01 07:00:00,70.857500,71.2625,66.1
2021-11-01 09:00:00,71.566667,71.0625,66.1
2021-11-01 11:00:00,68.812500,67.5750,66.1
2021-11-01 13:00:00,63.250000,63.7000,66.1
2021-11-01 15:00:00,65.589583,65.8750,66.1
...,...,...,...
2021-11-30 07:00:00,97.335714,97.8750,91.7
2021-11-30 09:00:00,98.860417,98.6250,91.7
2021-11-30 11:00:00,96.662500,96.3625,91.7


### Rolling average way

In [12]:
# Take 2-hour rolling average for each resolution
df_roll = df.groupby(['Resolution', df.index.normalize()]).rolling('2H').mean().reset_index(level=[0,1])\
.rename(columns={'Datetime':'Date'})

In [13]:
df_roll.head()

,Resolution,Date,Price
Datetime,,,
2021-11-01 07:00:00,10MIN,2021-11-01,70.000000
2021-11-01 07:10:00,10MIN,2021-11-01,70.000000
2021-11-01 07:20:00,10MIN,2021-11-01,70.750000
2021-11-01 07:30:00,10MIN,2021-11-01,70.958333
2021-11-01 07:40:00,10MIN,2021-11-01,70.958333


In [14]:
# Split the final dataframe with 2-hour averages into 10MIN, 1H, 1D resolution
df_roll_10min = df_roll.loc[df_roll['Resolution']=='10MIN'].drop(['Resolution', 'Date'], axis=1)
df_roll_1H = df_roll.loc[df_roll['Resolution']=='1H'].drop('Resolution', axis=1)
df_roll_1D = df_roll.loc[df_roll['Resolution']=='D'].drop('Resolution', axis=1)

In [15]:
# Merge the 10min and 1h dataframes dataframes into a wide format
tensixty_roll_df = df_roll_10min.merge(df_roll_1H, left_index=True, right_index=True,how='left')\
.rename(columns={'Price_x':'10MIN', 'Price_y':'1H'}).ffill()

In [16]:
# Merge the day resolution into the dataframe with forward filling
final_roll_df = tensixty_roll_df.merge(df_roll_1D, right_index=True, left_on='Date', how='left')\
.drop(['Date_x', 'Date_y'], axis=1).rename(columns={'Price':'1D'}).ffill()

In [17]:
# View the final dataframe with 2-hour averages for 1D resolution
final_roll_df.head(20)

,10MIN,1H,1D
Datetime,,,
2021-11-01 07:00:00,70.000000,71.3250,66.1
2021-11-01 07:10:00,70.000000,71.3250,66.1
2021-11-01 07:20:00,70.750000,71.3250,66.1
2021-11-01 07:30:00,70.958333,71.3250,66.1
2021-11-01 07:40:00,70.958333,71.3250,66.1
2021-11-01 07:50:00,71.050000,71.3250,66.1
2021-11-01 08:00:00,71.020000,71.2625,66.1
2021-11-01 08:10:00,70.958333,71.2625,66.1
2021-11-01 08:20:00,70.914286,71.2625,66.1
